## Creating the data set using tapping information for 70000 samples

In [1]:
import synapseclient
import pandas as pd
import json
import csv
from scipy.stats import entropy
import numpy as np 

In [2]:

def TapPerformance(TapPD1):
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonLeft', 'TappedButtonId'] = 0
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonRight', 'TappedButtonId'] = 1
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonNone', 'TappedButtonId'] = 2
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonLeft'] = 0
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonRight'] = 1
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonNone'] = 2
    TapPD1 = Artifactfixed1(TapPD1)
    TapPerf= TapPD1.shape[0]
    return (TapPerf)

In [3]:
#Index=list()
def Artifactfixed(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TapCoordinate[item]==TapCont.TapCoordinate[item+1]:
            if TapCont.TappedButtonId[item]==2:
                Ind[item]=False
            elif TapCont.TappedButtonId[item+1]==2:
                Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed
          

In [4]:
def Artifactfixed1(TapCont):
    Ind =  [True] * TapCont.shape[0]
    for item in range(0,TapCont.shape[0]-1):
        if TapCont.TappedButtonId[item]==2:
            Ind[item]=False
    TapContArtFixed=TapCont[Ind]
    return TapContArtFixed

In [5]:
# Measures entropy of the time interval distribution.
def TapPerfEntropy(TapPD1):
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonLeft', 'TappedButtonId'] = 0
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonRight', 'TappedButtonId'] = 1
    TapPD1.loc[TapPD1.TappedButtonId == 'TappedButtonNone', 'TappedButtonId'] = 2
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonLeft'] = 0
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonRight'] = 1
    #TapPD1.TappedButtonId[TapPD1.TappedButtonId == 'TappedButtonNone'] = 2
    TapPD1 = Artifactfixed1(TapPD1)
    
    # The leftmost and rightmost bin edges
    a=TapPD1['TapTimeStamp']
    Temp=a.to_numpy()
    Int=Temp[2:-1]-Temp[1:-2]
    first_edge, last_edge = 0, Int.max()
    bin_edges = np.arange(first_edge, last_edge, step=0.01)
    #Estimating probabilities
    Counts, bin_edges=np.histogram(Int, bins=bin_edges)
    Prob=Counts/sum(Counts)
    En=entropy(Prob, base=2)
    return (En)

In [6]:
Cred = open("Credentials.txt","r",encoding='utf8')
lines=Cred.readlines()
username1=lines[0][:-1]
password1=lines[1][:-1]
Cred.close()



In [7]:
syn = synapseclient.Synapse()
syn.login(username1, password1)


## QUERY THE mPower PROJECT (syn4993293) FOR ALL OF THE TABLES
# See examples: https://python-docs.synapse.org/build/html/index.html 
# and https://www.synapse.org/#!Synapse:syn4993293/discussion/threadId=6785
with open('TappingTable.csv') as csvfile:
    TapTable=pd.read_csv(csvfile, delimiter=',')
with open('DemoTable.csv') as csvfile:
    DemoTable=pd.read_csv(csvfile, delimiter=',') 
#Using Patient column (healthCode) as index for the Data frames
#TapTableB = TapTable.set_index("healthCode", drop = False) 
#DemoTableB = DemoTable.set_index("healthCode", drop = False)    

      
#TapPD = list()
PathList = list()
LabelList = list()
#PatientType = list()
#PatientList = list()
#AgeList = list()
Perform =list()
EntropyTap = list()
for offset in range(0,70000,100):
    results = syn.tableQuery('SELECT * FROM syn5511439 LIMIT 100 OFFSET '+str(offset))
    file_map=syn.downloadTableColumns(results,['tapping_results.json.TappingSamples'])
    for file_handle_id, path in file_map.items():
        #TapPD.append(pd.read_json(path))
        if TapPerformance(pd.read_json(path))>10:
            PathList.append(path)
            LabelList.append(int(file_handle_id))
            Perform.append(TapPerformance(pd.read_json(path))) #Tap performance
            EntropyTap.append(TapPerfEntropy(pd.read_json(path)))
        #PatientList.append(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)])
        #DemoTableB.loc[(TapTableB['healthCode'][TapTableB['tapping_results.json.TappingSamples'] == int(file_handle_id)]),"age"]
        

Welcome, Catalina Rivera!



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



[WARNING] /Users/catalina/anaconda/envs/insight/lib/python3.8/site-packages/synapseclient/client.py:3481: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)



In [8]:
len(Perform)

69979

In [9]:
len(EntropyTap)

69979

In [10]:
len(LabelList)

69979

In [11]:
TapTable.columns
#Drop columns no needed
#TapTable=TapTable.drop(['ROW_ID', 'ROW_VERSION','recordId','createdOn','appVersion','phoneInfo','accel_tapping.json.items','tapping_results.json.ButtonRectLeft','tapping_results.json.ButtonRectRight','tapping_results.json.endDate','tapping_results.json.startDate','tapping_results.json.TappingViewSize','medTimepoint'], axis=1)
#Creating directory
d = {'tapping_results.json.TappingSamples':LabelList,'TapPerform':Perform,'EntropyTap':EntropyTap}
PerfInfo = pd.DataFrame(d)
#Merges TapTables with the PerfInfo and drops thos values in TapTable that are not defined in PerInfo
#TapTable=TapTable.merge(PerfInfo,left_on='tapping_results.json.TappingSamples', right_on='tapping_results.json.TappingSamples')
TapTable=TapTable.merge(PerfInfo)



In [12]:
#Save to a csv file
TapTable.head()
TapTable.to_csv(r'/Users/catalina/Dropbox/Insight/Parkinson Disease/TappingTableWithPerform&EntropyTap.csv')


In [13]:
TapTable.head()

,ROW_ID,ROW_VERSION,recordId,healthCode,createdOn,appVersion,phoneInfo,accel_tapping.json.items,tapping_results.json.ButtonRectLeft,tapping_results.json.ButtonRectRight,tapping_results.json.endDate,tapping_results.json.startDate,tapping_results.json.TappingSamples,tapping_results.json.TappingViewSize,medTimepoint,TapPerform,EntropyTap
0,82315,18,caf7a74b-0d2e-4c43-9233-ed5dc6e7b176,639e8a78-3631-4231-bda1-c911c1b169e5,1.425900e+12,"version 1.0, build 7",iPhone 6,5395782.0,"{{0, 191.5}, {104, 104}}","{{128, 191.5}, {104, 104}}",1.425900e+12,1.425900e+12,5395798.0,"{375, 603}",I don't take Parkinson medications,198,3.410725
1,82316,18,94c95572-b405-49ca-9eb0-7426254990d5,52fe366a-2a9f-4260-9fb1-0fbc637a6cf4,1.425930e+12,"version 1.0, build 7",iPhone 5s (GSM),5403976.0,"{{0, 156.5}, {104, 104}}","{{128, 156.5}, {104, 104}}",1.425930e+12,1.425930e+12,5403991.0,"{320, 504}",I don't take Parkinson medications,165,3.415324
2,82317,18,ebd99144-5b84-486e-818c-5f8daf8beebf,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5397281.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5397298.0,"{414, 672}",I don't take Parkinson medications,136,3.478756
3,82318,18,8e32cff2-c8f9-4047-af57-411a98fc9116,67bdd316-26fc-4fc7-8431-bf9f41a649dd,1.425930e+12,"version 1.0, build 7",iPhone 6 Plus,5395191.0,"{{0, 191.33333333333331}, {104, 104}}","{{128, 191.33333333333331}, {104, 104}}",1.425930e+12,1.425930e+12,5395211.0,"{414, 672}",I don't take Parkinson medications,157,3.288296
4,82319,18,69851454-05b2-44f1-9248-299ca684b801,be43841c-d034-4705-88da-87e3536163b2,1.425930e+12,"version 1.0, build 7",iPhone 6,5390401.0,"{{0, 189}, {104, 104}}","{{128, 189}, {104, 104}}",1.425930e+12,1.425930e+12,5390413.0,"{375, 603}",I don't take Parkinson medications,172,2.704786


In [14]:
DemoTable.shape

(6805, 33)

In [15]:
TapTable.shape

(69979, 17)